In [1]:
import requests
import time

# commander_input = "sheoldred the apocalypse"
commander_input = "RANDOM"
# num_ramp = 10
# num_draw = 10
# num_removal = 10
num_ramp = 15
num_draw = 5
num_removal = 10
enable_fetch = False
if commander_input == "RANDOM":
    num_ramp = 10
    num_draw = 10
    num_removal = 10

HEADERS = {
    "User-Agent": "deck-builder-bot/1.1",
    "Accept": "application/json"
}

def get_commander(name):
    url = "https://api.scryfall.com/cards/named"
    params = {
        "exact": name
    }

    r = requests.get(url, params=params, headers=HEADERS)
    r.raise_for_status()
    return r.json()


def fetch_all_cards(query):
    url = "https://api.scryfall.com/cards/search"
    params = {"q": query}
    print(query)
    cards = []
    card_index = 0
    while True:
        
        r = requests.get(url, params=params, headers=HEADERS)
        r.raise_for_status()
        data = r.json()
        card_index += len(data["data"]) 
        total_cards = data["total_cards"]
        percentage = float(card_index)/data["total_cards"] * 100
        print(f"current index {card_index}, total cards {total_cards}, percentage {percentage}")

        cards.extend(data["data"])

        if not data.get("has_more"):
            break

        # IMPORTANT: next_page already includes query params
        url = data["next_page"]
        params = None
        time.sleep(0.2) # dont want to get rate limited
    return cards


In [8]:
commander = ""
import random

if commander_input == "RANDOM":
    print("getting random commander")
    
    commander_list = fetch_all_cards((
        f"-is:playtest "
        f"f:commander "
        f"type:creature type:legendary "
        f"-is:gamechanger "
        f"-set:UNF -set:SUNF "
        f"-o:snow "
        f"(usd<30 or year<2010) "
    ))
    commander = commander_list[random.randint(0,len(commander_list))]
else:
    commander = get_commander(commander_input)

getting random commander
-is:playtest f:commander type:creature type:legendary -is:gamechanger -set:UNF -set:SUNF -o:snow (usd<30 or year<2010) 
current index 175, total cards 2728, percentage 6.414956011730205
current index 350, total cards 2728, percentage 12.82991202346041
current index 525, total cards 2728, percentage 19.244868035190617
current index 700, total cards 2728, percentage 25.65982404692082
current index 875, total cards 2728, percentage 32.07478005865102
current index 1050, total cards 2728, percentage 38.489736070381234
current index 1225, total cards 2728, percentage 44.90469208211144
current index 1400, total cards 2728, percentage 51.31964809384164
current index 1575, total cards 2728, percentage 57.734604105571854
current index 1750, total cards 2728, percentage 64.14956011730204
current index 1925, total cards 2728, percentage 70.56451612903226
current index 2100, total cards 2728, percentage 76.97947214076247
current index 2275, total cards 2728, percentage 83.3

In [9]:
color_identity = commander["color_identity"]
print(commander['name'], color_identity)  # e.g. ['G', 'U', 'B', 'W']
def color_identity_query(ci):
    if not ci:
        return "ci=0"   # colorless commanders (Kozilek, etc.)
    return f"ci<={''.join(ci)}"
cid_string = color_identity_query(color_identity)
print(cid_string)

Iraxxa, Empress of Mars ['R']
ci<=R


In [10]:
QUERIES = {
    "ramp" : f"usd<10 function:ramp {cid_string} -is:playtest -name:\"Sol Ring\" cmc<5 -type:land -set:UNF f:commander -is:gamechanger -o:snow",
    "draw" : f"usd<10 function:draw {cid_string} -is:playtest cmc<5 -type:land -set:UNF f:commander -is:gamechanger -o:snow",
    "removal" : f"usd<10 function:removal {cid_string} -is:playtest -type:land -set:UNF f:commander -is:gamechanger -o:snow"
}

ramp_cards    = fetch_all_cards(QUERIES["ramp"])
draw_cards    = fetch_all_cards(QUERIES["draw"])
removal_cards = fetch_all_cards(QUERIES["removal"])

usd<10 function:ramp ci<=R -is:playtest -name:"Sol Ring" cmc<5 -type:land -set:UNF f:commander -is:gamechanger -o:snow
current index 175, total cards 394, percentage 44.41624365482234
current index 350, total cards 394, percentage 88.83248730964468
current index 394, total cards 394, percentage 100.0
usd<10 function:draw ci<=R -is:playtest cmc<5 -type:land -set:UNF f:commander -is:gamechanger -o:snow
current index 175, total cards 466, percentage 37.553648068669524
current index 350, total cards 466, percentage 75.10729613733905
current index 466, total cards 466, percentage 100.0
usd<10 function:removal ci<=R -is:playtest -type:land -set:UNF f:commander -is:gamechanger -o:snow
current index 175, total cards 1750, percentage 10.0
current index 350, total cards 1750, percentage 20.0
current index 525, total cards 1750, percentage 30.0
current index 700, total cards 1750, percentage 40.0
current index 875, total cards 1750, percentage 50.0
current index 1050, total cards 1750, percentage

In [31]:
import random

def pick_random(cards, n):
    if len(cards) < n:
        raise ValueError(f"Not enough cards to sample {n}")
    return random.sample(cards, n)

def reroll_until_disjoint(ramp_cards, draw_cards, removal_cards,
                          num_ramp, num_draw, num_removal,
                          max_attempts=50):

    for attempt_number in range(max_attempts):
        chosen_ramp    = pick_random(ramp_cards, num_ramp)
        chosen_draw    = pick_random(draw_cards, num_draw)
        chosen_removal = pick_random(removal_cards, num_removal)

        names = (
            [c["name"] for c in chosen_ramp] +
            [c["name"] for c in chosen_draw] +
            [c["name"] for c in chosen_removal]
        )

        if len(names) == len(set(names)):
            return chosen_ramp, chosen_draw, chosen_removal
        print(f"Rerolling, attempt {attempt_number}")

    raise RuntimeError("Could not find disjoint role assignment")


chosen_ramp, chosen_draw, chosen_removal = reroll_until_disjoint(ramp_cards, draw_cards, removal_cards, num_ramp, num_draw, num_removal)

In [13]:
COLOR_TO_BASIC = {
    "W": "Plains",
    "U": "Island",
    "B": "Swamp",
    "R": "Mountain",
    "G": "Forest"
}

def get_basic_lands(color_identity, total_basics=16):
    if not color_identity:
        return ["Wastes"] * total_basics

    basics = []
    per_color = total_basics // len(color_identity)
    remainder = total_basics % len(color_identity)

    for c in color_identity:
        basics.extend([COLOR_TO_BASIC[c]] * per_color)

    for i in range(remainder):
        basics.append(COLOR_TO_BASIC[color_identity[i]])

    return basics

def multicolor_land_query(ci):
    return (
        f"type:land -type:basic legal:commander -is:gamechanger -is:playtest "
        f"ci<={''.join(ci)} "
        f"o:/add .*[{''.join(ci)}].*[{''.join(ci)}]/"
    )


FETCH_QUERY = (
    "type:land legal:commander "
    "(o:search or o:sacrifice) -is:gamechanger -is:playtest "
    "-o:add"
)

multicolor_lands = fetch_all_cards(multicolor_land_query(color_identity))
# since we dont have consistent dual lands, I decided to disable for now
fetch_lands = []
if enable_fetch:
    fetch_lands = fetch_all_cards(FETCH_QUERY)


type:land -type:basic legal:commander -is:gamechanger -is:playtest ci<=R o:/add .*[R].*[R]/
current index 74, total cards 74, percentage 100.0


In [14]:
import random
print('getting land base')
def pick(cards, n):
    return random.sample(cards, min(n, len(cards)))


chosen_fetches    = pick(fetch_lands, 10)

land_base = []
basic_lands = []
chosen_multicolor = []
if len(color_identity) == 1:
    print('getting only basics')
    if enable_fetch:
        basic_lands       = get_basic_lands(color_identity, 28)
    else:
        basic_lands       = get_basic_lands(color_identity, 38)
elif len(color_identity) == 2:
    print('getting multi lands and basics')
    if enable_fetch:
        basic_lands       = get_basic_lands(color_identity, 16)
        chosen_multicolor = pick(multicolor_lands, 12)
    else:
        basic_lands       = get_basic_lands(color_identity, 21)
        chosen_multicolor = pick(multicolor_lands, 17)
elif len(color_identity) == 3:
    print('getting multi lands and basics')
    if enable_fetch:
        basic_lands       = get_basic_lands(color_identity, 8)
        chosen_multicolor = pick(multicolor_lands, 20)
    else:
        basic_lands       = get_basic_lands(color_identity, 13)
        chosen_multicolor = pick(multicolor_lands, 25)
elif len(color_identity) >= 4:
    print('getting multi lands and basics')
    if enable_fetch:
        basic_lands       = get_basic_lands(color_identity, 4)
        chosen_multicolor = pick(multicolor_lands, 24)
    else:
        basic_lands       = get_basic_lands(color_identity, 9)
        chosen_multicolor = pick(multicolor_lands, 29)

# Basics (names only)
for b in basic_lands:
    land_base.append({"name": b})

# Multicolor lands
land_base.extend(chosen_multicolor)

# Fetch lands
land_base.extend(chosen_fetches)

getting land base
getting only basics


In [15]:
SOL_RING = "Sol Ring"

In [25]:
def random_filler_query(ci):
    ci_filter = f"ci<={''.join(ci)}" if ci else "ci=0"
    # one printing of Red Herring is illegal 
    return (
        f"legal:commander "
        f"{ci_filter} "
        f"-type:land "
        f"-is:funny "
        f"-set:UNF "
        f"-set:SUNF "
        f"-o:snow "
        f"-is:playtest "
        f"-type:conspiracy "
        f"-type:emblem "
        f"-name:\"Sol Ring\" "
        f"-name:\"Red Herring\" "
        f"(usd<=1 or year<=2010)"
    )

print("fetching filler cards")
filler_cards = fetch_all_cards(random_filler_query(color_identity))
unique_by_name = {}
for card in filler_cards:
    unique_by_name.setdefault(card["name"], card)

unique_cards = list(unique_by_name.values())

random_cards = random.sample(
    unique_cards,
    min(30, len(unique_cards))
)


fetching filler cards
legal:commander ci<=R -type:land -is:funny -set:UNF -set:SUNF -o:snow -is:playtest -type:conspiracy -type:emblem -name:"Sol Ring" -name:"Red Herring" (usd<=1 or year<=2010)
current index 175, total cards 6043, percentage 2.89591262617905
current index 350, total cards 6043, percentage 5.7918252523581
current index 525, total cards 6043, percentage 8.68773787853715
current index 700, total cards 6043, percentage 11.5836505047162
current index 875, total cards 6043, percentage 14.47956313089525
current index 1050, total cards 6043, percentage 17.3754757570743
current index 1225, total cards 6043, percentage 20.27138838325335
current index 1400, total cards 6043, percentage 23.1673010094324
current index 1575, total cards 6043, percentage 26.063213635611447
current index 1750, total cards 6043, percentage 28.9591262617905
current index 1925, total cards 6043, percentage 31.85503888796955
current index 2100, total cards 6043, percentage 34.7509515141486
current index 

In [32]:
deck = []
deck.append(commander)
# Core spells
deck.extend(chosen_ramp)
deck.extend(chosen_draw)
deck.extend(chosen_removal)

# Lands
deck.extend(land_base)

# Random filler
deck.extend(random_cards)

# Sol Ring
deck.append({"name": SOL_RING})

In [33]:
from collections import Counter

counts = Counter(c["name"] for c in deck)

for name, count in sorted(counts.items()):
    print(f"{count} {name}")

1 Acolyte Hybrid
1 Adventurer's Airship
1 Adventuring Gear
1 Akki Scrapchomper
1 Bag of Holding
1 Bargaining Table
1 Barrin's Codex
1 Blazing Effigy
1 Bottle Gnomes
1 Brightstone Ritual
1 Caterwauling Boggart
1 Change of Fortune
1 Char
1 Dismissive Pyromancer
1 Dreamstone Hedron
1 Ebony Fly
1 Extra Arms
1 Fall of Cair Andros
1 Fiery Conclusion
1 Fiery Encore
1 Fire Dragon
1 Flame Jet
1 Glimpse the Impossible
1 Gold-Forged Sentinel
1 Harvesttide Infiltrator // Harvesttide Assailant
1 Heat Shimmer
1 Henge Guardian
1 Illusionary Mask
1 Inner-Flame Acolyte
1 Invasion of Kaldheim // Pyre of the World Tree
1 Iraxxa, Empress of Mars
1 J. Jonah Jameson
1 Kilnmouth Dragon
1 Kinetic Augur
1 Last-Ditch Effort
1 Marauding Maulhorn
1 Markov Warlord
38 Mountain
1 Ornithopter of Paradise
1 Pilgrim of the Fires
1 Pyroclastic Hellion
1 Roc of Kher Ridges
1 Rodeo Pyromancers
1 Sabretooth Tiger
1 Sarkhan, Fireblood
1 Sculpting Steel
1 Sibylline Soothsayer
1 Sol Ring
1 Song of Blood
1 Sparkhunter Masticor

In [28]:
print(f"\nchosen_ramp {num_ramp}\n########################")
for card in chosen_ramp:
    print(card['name'])
print(f"\nchosen_draw {num_draw}\n########################")
for card in chosen_draw:
    print(card['name'])
print(f"\nchosen_removal {num_removal}\n########################")

for card in chosen_removal:
    print(card['name'])


chosen_ramp 10
########################
Smashing Success
Swashbuckler Extraordinaire
Nahiri's Lithoforming
Spectral Searchlight
Bandit's Haul
Improvised Weaponry
Magda, Brazen Outlaw
Breeches, Eager Pillager
Burnished Hart
Zookeeper Mechan

chosen_draw 10
########################
Well of Lost Dreams
Bandit's Haul
Credit Voucher
Goblin Surveyor
Snazzy Aether Homunculus
Wedding Invitation
Red Herring
Collective Defiance
Burning-Tree Vandal
Phyrexian Furnace

chosen_removal 10
########################
Ranger's Firebrand
Shenanigans
Goretusk Firebeast
Chandra, Bold Pyromancer
Tower of Calamities
Barrage Tyrant
Chaos Maw
Fanged Flames
Detonate
Granite Shard
